<a href="https://colab.research.google.com/github/Yutongzhang20080108/Using-a-Combination-of-DPO-and-RAG-to-Improve-the-Chinese-Poetry-Generation/blob/main/Test_between_Human_and_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [3]:
from langchain_openai import ChatOpenAI

generate_model = ChatOpenAI(model="gpt-4o", base_url="https://api.chatanywhere.tech/v1")
grade_model = ChatOpenAI(model="gpt-4", base_url="https://api.chatanywhere.tech/v1")

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
generate_system_template = "你是一个善于写中国古代诗歌的诗人，接下来我会给你提供一个题目，请你围绕这个题目写一首古诗。在生成的过程中请你注意中国古代诗歌的格式要求，并且请你表达尽可能多的情感与含义。在最后输出结果时只需要生成诗歌的主体部分"
generate_prompt_template = ChatPromptTemplate.from_messages(
    [("system", generate_system_template), ("user", "{text}")]
)

grade_system_template = "你是一个中国古代诗歌评论家，接下来我会给你一首诗歌，请你依据以下标准来评价这首诗歌的得分。评分标准为：①格式。生成的唐诗是否遵循写作规则,尤其注意对仗、句数、字数、平仄（0至5分）；②创新性。生成的唐诗是否抄袭现成的诗句，是否具有创新性（0至5分）；③相关性。生成的唐诗与给定主题是否相关（0至5分）；④美学。生成的唐诗是否符合中国诗词的审美，包括寓情于物等（0至5分）；⑤整体。从整体情况评价生成的唐诗（0至10分）.请你在最后给出这几项的得分，并且你不需要给出评判的过程和任何文字只需要输出最后的5个数字。"
grade_prompt_template = ChatPromptTemplate.from_messages(
    [("system", grade_system_template), ("user", "{text}")]
)

In [11]:
generate_chain = generate_prompt_template | generate_model | parser
grade_chain = grade_prompt_template | grade_model | parser

In [13]:
theme = ["青春", "梦想", "高山", "深海", "星空", "失落", "爱情", "孤独", "希望", "迷茫",
    "成长", "离别", "重逢", "夕阳", "清晨", "冬雪", "春花", "夏雨", "秋叶", "往事",
    "忧伤", "欢乐", "旅行", "归宿", "朋友", "家庭", "传统", "创新", "自由", "约束",
    "城市", "乡村", "河流", "山谷", "荒漠", "林间", "夜晚", "黎明", "阴雨", "晴天",
    "风景", "人群", "孤单", "热闹", "平静", "激情", "压力", "解脱", "寻找", "发现",
    "失去", "获得", "旧时光", "新生", "悲伤", "喜悦", "分手", "团聚", "哲理", "纷争",
    "和平", "冒险", "安全", "学习", "教育", "成就", "失败", "胜利", "竞争", "合作",
    "梦魇", "幻想", "现实", "逃避", "面对", "变化", "恒常", "追忆", "遗忘", "诗歌",
    "绘画", "音乐", "舞蹈", "雕塑", "建筑", "文学", "历史", "未来", "科技", "自然",
    "灾害", "恢复", "衰退", "繁荣", "衰弱", "强健", "疾病", "治愈", "传说", "神话"]

score = []
result = []
for each in theme:
  poetry = generate_chain.invoke(each)
  result.append(poetry)
  score.append(grade_chain.invoke(poetry))
print(score)
print(result)

['4.5 4.5 4 4 8.5', '5 5 4 5 9', '5 5 5 5 10', '4.5, 4, 4.5, 4.5, 9', '5 4 4 5 9', '5 4 5 4 8', '5 4 4 5 8', '4 4 4 5 8', '5 5 5 5 10', '4.5 4 4.5 4.5 8.5', '5 5 5 5 10', '5 5 5 4 9', '5 5 5 5 10', '4, 4, 4, 5, 8', '5 5 4 4 8', '4.5 4 5 4.5 9', '5 5 5 5 10', '5 5 5 5 10', '5 5 5 5 10', '4, 4, 5, 5, 8', '4 4 4 4 8', '5 4 4 4 8', '5 5 5 5 10', '4.5 4.5 4 4.5 8.5', '5 4 4 4 8', '5 5 5 5 10', '5 5 5 5 10', '4,4,4,4,8', '5 5 4 4 8', '4 4 4 4 8', '4 4 4 4 8', '5 4 5 4 9', '5 4 5 5 9', '5 5 5 5 10', '5 5 5 5 10', '4.5 4 5 4.5 8.5', '4,5,4,5,9', '5 5 5 4 9', '5 5 5 5 10', '4, 4, 3, 4, 7', '5 5 5 5 10', '5 5 4 5 9', '5 5 5 5 10', '5 5 4 4 8', '5 5 5 5 10', '5 5 5 5 10', '5 5 5 5 10', '4 5 4 4 8', '5 5 5 5 10', '4.5 5 5 4.5 9', '5 5 5 5 10', '4 5 5 4 8', '5 5 5 5 10', '5 5 5 4 9', '5 5 5 5 10', '5 4 5 5 9', '4 4 4 4 8', '5 4 5 5 9', '5 5 4 4 8', '5 4 5 4 8', '5 5 5 5 10', '5 5 5 5 9', '5 5 5 5 10', '5 5 5 5 10', '4 4 4 5 8', '5 5 5 5 10', '5 5 4 5 9', '5 4 5 4 9', '5 5 5 5 10', '4 4.5 4 4.5 8.5'